# Model manipulation

Performing state estimation with larger models or actual measurements may lead to divergence or result in unrealistic outcomes.
This could be due to incorrect model parameters.

To improve the results, multiple methods can be used:
- Add generators with $P=Q=0$ at buses that do not have any appliances (passive nodes). These generators can compensate for power mismatches.
- Split branches: Some branches may have incorrect parameters, resulting in divergence. By splitting these branches into two generators/loads, the incorrect parameters can be compensated.
- To debug diverging networks, it may be useful to split the network into smaller parts and run the state estimation on these parts, checking which parts converge and which do not.

This package provides methods to manipulate an existing PGM dataset, which are described in this notebook.

## Data Setup

We will create the dataset and run the measurement simulation analog to the `state-estimation` notebook.

In [1]:
from cgmes2pgm_converter import CgmesDataset
from cgmes2pgm_converter.common import Profile

base_url = "http://localhost:3030/dataset_name"
dataset = CgmesDataset(
    base_url=base_url,
    # cim_namespace="http://iec.ch/TC57/2013/CIM-schema-cim16#",
    cim_namespace="http://iec.ch/TC57/CIM100#",
    graphs={
        Profile.OP: base_url + "/op",
        Profile.MEAS: base_url + "/meas",
    },
)

Run Measurement Simulation:

In [2]:
from cgmes2pgm_suite.measurement_simulation import MeasurementBuilder
from cgmes2pgm_suite.config import MeasurementSimulationConfigReader

reader = MeasurementSimulationConfigReader("meas_ranges.yaml")
config = reader.read()

builder = MeasurementBuilder(dataset, config)
builder.build_from_sv()

INFO     :: Building Voltage Measurements: 0.4082028865814209 seconds
INFO     :: Building Power Measurements: 2.3336668014526367 seconds


## Splitting the network

If the state estimation diverges, it may be due to incorrect parameters of the branches, missing network elements or wrong measurements. 
To localize the problem, the network can be split into smaller parts using the following methods.

### Splitting branches

Specific branches can be replaced by two loads. The branches are specified using their `cim:IdentifiedObject.name` attribute.

> **Hint:**  
> By setting `add_sources=True` the split branches are replaced with two sources instead of two generators.
> If multiple islands are created by splitting the branches and some islands don't have a source,
> this option allows that all islands can be calculated.
> 
> By setting `StesOptions.compute_islands_separately=True` the islands are calculated separately.



In [3]:
from cgmes2pgm_converter import CgmesToPgmConverter, ConverterOptions
from cgmes2pgm_converter.common import NetworkSplittingOptions
from cgmes2pgm_suite.state_estimation import (
    StesOptions,
    StateEstimationWrapper,
)

splitting_options = NetworkSplittingOptions(
    enable=True,
    cut_branches=["L-12-13-1", "L-9-10-1", "L-9-11-1"],
    add_sources=False,
)

options = ConverterOptions(
    network_splitting=splitting_options,
)

converter = CgmesToPgmConverter(datasource=dataset, options=options)
input_data, extra_info = converter.convert()


state_estimation_wrapper = StateEstimationWrapper(
    input_data,
    extra_info,
    stes_options=StesOptions(),
)
result = state_estimation_wrapper.run()
print(result)

INFO     :: State Estimation: 0.0013873577117919922 seconds
---------------------------------------------
State Estimation Results
---------------------------------------------
Run name            network
Converged           True
---------------------------------------------
J                    907.59
E(J)                 854.00
E(J) ± 3σ           [730.02; 977.98]
Redundancy           3.56
---------------------------------------------
Bad measurements U   0
Bad measurements P   0
Bad measurements Q   2
---------------------------------------------



### Split by Substation

Another option is to split the network by substation: All branches to the provided substations are removed and replaced with two generators/loads. Branches within the substations are not removed.

If no substation names are provided, all branches between the substations are removed. This allows for calculating each substation separately.

In [4]:
splitting_options = NetworkSplittingOptions(
    enable=True,
    cut_substations=["S4", "S6"],
    add_sources=False,
)

options = ConverterOptions(
    network_splitting=splitting_options,
)

converter = CgmesToPgmConverter(datasource=dataset, options=options)
input_data, extra_info = converter.convert()

## Reconnecting Branches

The Parameter `StesOptions.reconnect_branches` allows for automatically reconnecting branches, that have been split during conversion. This mode runs the state estimation multiple times and reconnects all branches unless they cause the state estimation to diverge.

For this process, `StateEstimationWrapper` provides a list of all StateEstimationResults.
In addition, the branches, that could not be reconnected are printed to the console.

> **Hint:**  
> This mode requires `NetworkSplittingOptions.add_sources` to be set to `True` during PGM conversion.

## Measurement Substitution

### Passive Nodes

To improve the results, additional generators/loads can be added to nodes that currently do not have any generators, loads or sources. The added generators/loads can compensate for power mismatches or missing network elements by injecting or consuming power at those nodes.

In [5]:
from cgmes2pgm_converter.common import (
    MeasurementSubstitutionOptions,
    PassiveNodeOptions,
)

measurement_substitution_options = MeasurementSubstitutionOptions(
    passive_nodes=PassiveNodeOptions(
        enable=True,
        sigma=3,  # MW/MVAr
    )
)
options = ConverterOptions(
    measurement_substitution=measurement_substitution_options,
)
converter = CgmesToPgmConverter(datasource=dataset, options=options)
input_data, extra_info = converter.convert()

### Q-Measurements from Current (I-)Measurements

For now, PGM does not support current measurements.
For shunts, Q-measurements can be derived from the current measurements.

Assuming no active power flow to the shunt, the reactive power is given by $Q = \sqrt{3} \cdot U \cdot I$

>**Hint:**
> The node connected to the shunt needs to have a voltage measurement.

In [6]:
from cgmes2pgm_converter.common import QFromIOptions

measurement_substitution_options = MeasurementSubstitutionOptions(
    imeas_used_for_qcalc=QFromIOptions(
        enable=True,
        sigma=3,  # MW/MVAr
    )
)
options = ConverterOptions(
    measurement_substitution=measurement_substitution_options,
)

converter = CgmesToPgmConverter(datasource=dataset, options=options)
input_data, extra_info = converter.convert()